## Import 

In [2]:
import os

# ignore les deprecated de tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, Activation, Dropout, Flatten, Dense

import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
import numpy as np

C:\Users\utilisateur\anaconda3\envs\cnn\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\utilisateur\anaconda3\envs\cnn\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\utilisateur\anaconda3\envs\cnn\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\utilisateur\anaconda3\envs\cnn\lib\site-packages\ten

## Telechargement du ZIP

In [ ]:
from modules.loader import Loader

loader = Loader(
    "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz",
    '../datas/ZIP/',
    extraction_target='../datas/RAW/cifar-10'
)
loader.ensure_data_loaded()

In [1]:
from extract_cifar import extract_cifar_10

cifar_path = '..' + os.sep + 'datas' + os.sep + 'RAW' + os.sep + 'cifar-10'
cifar_path += os.sep + os.listdir('..' + os.sep + 'datas' + os.sep + 'RAW' + os.sep + 'cifar-10')[0]

curated_path = '..' + os.sep + 'datas' + os.sep + 'CURATED' + os.sep + 'taillequicompte'

if len(os.listdir(curated_path)) == 0 or not os.path.exists(curated_path):
    extract_cifar_10(cifar_path, curated_path)

## Extraction d'un jeu de donnees

In [ ]:
from modules.splitting import Splitting

labels_array = ['apple', 'bee']

TRAIN_DATA_DIR = Splitting.copie_dossiers(
    '../datas/RAW/train',
    labels_array,
    500,
    explorer=False
)

print(TRAIN_DATA_DIR)

## Chargement des images

In [ ]:
image_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

# Taille d'image
TRAIN_IMAGE_SIZE = 32
# NB d'images envoyées à la fois
TRAIN_BATCH_SIZE = 64

train_generator = image_data_generator.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
    batch_size=TRAIN_BATCH_SIZE,
    class_mode='categorical',
    subset='training')
 
validation_generator = image_data_generator.flow_from_directory(
    TRAIN_DATA_DIR, # same directory as training data
    target_size=(TRAIN_IMAGE_SIZE, TRAIN_IMAGE_SIZE),
    batch_size=TRAIN_BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

## Creation du modele

- convolution
- dense
- pooling

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same', input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dense(64, activation='elu'))
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same'))
model.add(Dense(128, activation='elu'))
model.add(Dropout(0.2))

model.add(Conv2D(128, kernel_size=3, activation='elu', kernel_initializer='he_uniform', padding='same'))

#Toujours à la fin
model.add(Flatten())
model.add(Dense(2, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es_callback = EarlyStopping(monitor='val_loss', patience=15)
training = model.fit(train_generator, epochs=40, callbacks=[es_callback], validation_data=validation_generator, shuffle=False)


In [ ]:
plt.plot(training.history['accuracy'], color='red', label='Training accuracy')
plt.plot(training.history['val_accuracy'],  color='green', label='Validation accuracy')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()
plt.ylim((0,1.01))

plt.show()

In [ ]:
plt.plot(training.history['loss'], color='red', label='Training loss')
plt.plot(training.history['val_loss'],  color='green', label='Validation loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.legend()

plt.show()

## Sauvegarde du modele

In [ ]:
model_name = 'model_apple_bee.h5'

model.save(model_name)

model.summary()

## Affichage des couches du modele

In [ ]:
from modules.observation_modele import plot_layer

img = "../datas/RAW/train/apple/0020.png"
plt.imshow(plt.imread(img))

plot_layer(model, img, range(8))

In [ ]:
from modules.observation_modele import plot_layer

img = "../datas/RAW/train/bee/0103.png"
plt.imshow(plt.imread(img))

plot_layer(model, img, range(8))